# Język Python - Wykład 5

# Monkey-patching

    from mock import MagicMock
    thing = ProductionClass()
    thing.method = MagicMock(return_value=3)
    thing.method(3, 4, 5, key='value')

    thing.method.assert_called_with(3, 4, 5, key='value')

In [3]:
import os
os.urandom(20)

b'\x15\x0e<\xea\xe0Kz`\xe9\xafX\xd2%!\xaf\xad[W\x1b5'

In [4]:
from unittest.mock import patch

def abc_urandom(length):
    return 'abc' + os.urandom(length)

print(os.urandom(4))
with patch('os.urandom', return_value='pumpkins') as abc_urandom_function:
    print(abc_urandom(4))
    print(os.urandom(4))
print(os.urandom(4))

b'\xa3rK\xe4'
abcpumpkins
pumpkins
b'l\xb7\xd0\x9c'


In [5]:
def new_open(a,b):
    print("opening ", a)
    return None
    
with patch('builtins.open', new_open):
    f = open("1.txt","r")

opening  1.txt


# Mixin

In [1]:
class FooMixin():
    def foo(self):
        print("fooo")

In [2]:
class Point2D():
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def r(self):
        return (self.x**2+self.y**2)**0.5

In [3]:
p1 = Point2D(2,2)
p1.r()

2.8284271247461903

In [4]:
class Point2Dmix(FooMixin,Point2D):
    pass
p2 = Point2Dmix(3,3)
p2.foo()
p2.r()

fooo


4.242640687119285

https://docs.python.org/3.5/library/socketserver.html

In [5]:
import socketserver
class TCPServer(socketserver.TCPServer):
    pass
s = dir(TCPServer)

In [6]:
class ThreadingTCPServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
  pass
class ForkingTCPServer(socketserver.ForkingMixIn, socketserver.TCPServer):
  pass
t = dir(ForkingTCPServer)
set(t) - set(s)

{'active_children', 'collect_children', 'max_children'}

In [7]:
class ThreadingUDPServer(socketserver.ThreadingMixIn, socketserver.UDPServer):
  pass
class ForkingUDPServer(socketserver.ForkingMixIn, socketserver.UDPServer):
  pass

# Metaclass

(Opracowanie oparte na świetnym wpisie StackOverflow @e-satis: http://stackoverflow.com/questions/100003/what-is-a-metaclass-in-python)

Tim Peters (ten od PEP 20 - Zen of Python oraz od timsort!)


<img src="files/L7_img/tim.jpg">


powiedział:

***Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don't (the people who actually need them know with certainty that they need them, and don't need an explanation about why).***

## Klasy jako obiekty

Definicje klas służą nam zwykle jako kod, za pomocą którego tworzone są obiekty:


In [6]:
class ObjectCreator():
    pass

In [7]:
print(ObjectCreator())

Ale klasy są także same w sobie... obiektami!

In [8]:
print(ObjectCreator)

<class '__main__.ObjectCreator'>


Oznacza to, że obiekt reprezentujący klasę ma umiejętność samemu tworzenia (instancjonowania) innych obiektów. Z tego powodu jest właśnie klasą!

Ale skoro każda klasa jest obiektem to oczywiście możemy:
 
 * przypisać ją do zmiennej,
 * skopiować ją,
 * dodać do niej atrybut,
 * przekazać ją jako argument funkcji.


In [9]:
FooCreator = ObjectCreator
print(FooCreator())

In [10]:
print(hasattr(ObjectCreator, 'bar'))

False


In [11]:
ObjectCreator.bar = 'baz'

In [12]:
print(hasattr(ObjectCreator, 'bar'))

True


In [13]:
def echo(x):
    print(x)

echo(ObjectCreator)

<class '__main__.ObjectCreator'>


## Dynamiczne tworzenie klas

Ponieważ klasy są obiektami, można je tworzyć dynamicznie (w run-time):

In [14]:
def choose_class(name):
    if name == 'foo':
        class Foo():
            pass
        return Foo # return the class, not an instance
    else:
        class Bar():
            pass
        return Bar

In [15]:
print(choose_class('foo'))
print(choose_class('bar'))

<class '__main__.choose_class.<locals>.Foo'>
<class '__main__.choose_class.<locals>.Bar'>


In [16]:
print(choose_class('bar')())

<__main__.choose_class.<locals>.Bar object at 0x7f8abf55f0f0>


To podejście nie jest jednak w pełni dynamiczne, bo nadal jest gdzieś explicite napisany kod definiujący tę klasę.

Ponieważ klasy są obiektami, to także i one muszą być przez coś tworzone. Gdy piszemy deklarację klasy i używamy słowa kluczowego **class**, interpreter Pythona wywołuje pewny kod, który tworzy tę klasę i zapisuje ją pod podaną nazwą w namespace (podobnie jak dekorator, wywołuje kod i zwraca obiekt callable do podstawienia pod daną nazwą).

W Pythonie tworzenie obiektu klasy odbywa się explicite poprzez użycie funkcji type.

In [17]:
print(type(1))

<class 'int'>


In [18]:
print(type("1"))

<class 'str'>


In [19]:
print(type(ObjectCreator()))

<class '__main__.ObjectCreator'>


In [20]:
print(type(ObjectCreator))

<class 'type'>


In [21]:
class MyShinyClass():
    pass
print(MyShinyClass)

<class '__main__.MyShinyClass'>


In [22]:
print(type('MyShinyClass', (), {}))

<class '__main__.MyShinyClass'>


In [23]:
ms = type('MyShinyClass', (), {})
print(ms())

In [24]:
class Foo():
    bar = True

class FooChild(Foo):
    baz = False

In [25]:
FooChild = type('FooChild', (Foo,), {'baz': False})
ob = FooChild()
print(ob.bar, ob.baz)

True False


## Czym są metaklasy?

**Metaklasa jest tym czymś, co tworzy obiekt reprezentujący klasę w Pythonie.**

Można to sobie wyobrazić w ten sposób:

```
MyClass = MetaClass()
MyObject = MyClass()
```

Prównajmy to z tym, co już wiemy o tworzeniu klas:

```
MyClass = type('MyClass', (), {})
```

Z tego wynika, że samo **type** jest metaklasą w Pythonie. Co więcej jest ono domyślną metaklasą używaną zawsze standardowo, gdy Python stara się utworzyć jakąś klasę z jej deklaracji i definicji.

**Dlaczego type() jest przeciążone?**

Ze względu na zaszłości językowe i kompatybilność wsteczną

**Dlaczego type() jest pisane z małej litery skoro to metaklasa?**

Ze względu na konwencję innych typów, int, str, unicode (to było pisane z małej litery w Pythonie).

In [26]:
age = 35
print(age.__class__)

<class 'int'>


In [27]:
name = 'bob'
print(name.__class__)

<class 'str'>


In [28]:
def foo(): pass
print(foo.__class__)

<class 'function'>


In [29]:
print(ObjectCreator().__class__)

<class '__main__.ObjectCreator'>


W takim razie - jaka jest klasa obiektu reprezentującego klasę klasy? 

In [30]:
print(ObjectCreator.__class__.__class__)

<class 'type'>


In [31]:
print(age.__class__.__class__)

<class 'type'>


In [32]:
print(name.__class__.__class__)

<class 'type'>


A więc podsumowując - metaklasa to nic innego jak "fabryka klas". **type** jest standardową metaklasą w Pythonie, ale Python pozwala nam ją podmienić jeśli chcemy!

## ``metaclass``

Za pomocą parametru  ``metaclass`` można wskazać której metaklasy ma Python użyć do tworzenia tej klasy.

```
class Foo(metaclass=some_metaclass):
    pass
```

Uwaga - jest to pewien hack języka. W momencie deklaracji klasy **class Foo()** tak naprawdę obiekt tej klasy nie jest jeszcze tworzony. Najpierw przeszukiwana jest jej definicja czy znajduje się tam **``metaclass``** - jeśli tak, jest dopiero wtedy używana do utworzenia obiektu/klasy Foo. Jeśli nie ma - użyty zostanie **type**.

A co z dziedziczeniem?

```
class Bar(Foo):
    pass
```

Python sprawdzi:

 * Czy Bar zawiera parametr ``metaclass``? 
 * Czy dowolny rodzic klasy (np. Foo) zawiera parametr ``metaclass``? 
 * Czy moduł zawiera atrybut ``metaclass``?
 * Jeśli żadne z powyższych się nie powiodło - użyje **type**.

## Tworzenie własnej metaklasy

Metaklas możemy potrzebować, jeśli chcemy modyfikować obiekty reprezentujące klasy podczas ich tworzenia. Jest to szczególnie pomocne przy tworzeniu API. Django ORM (modele) zostały utworzone właśnie z użyciem metaklas.

Dla uproszczenia przyjmijmy, że chcemy utworzyć metaklasę, która zapewni, że wszystkie atrybuty klasy będą miały w nazwie tylko WIELKIE LITERY, niezależnie od tego jak programista napiszę definicję klasy.

**Metaklasą** (pomimo nazwy) może być cokolwiek, co da się wywołać (jest callable) i przyjmie argumenty takie same jak **type(name, bases, dict)**

In [8]:
# the metaclass will automatically get passed the same argument
# that you usually pass to `type`
def upper_attr(future_class_name, future_class_parents, future_class_attr):
  """
    Return a class object, with the list of its attribute turned 
    into uppercase.
  """

  # pick up any attribute that doesn't start with '__' and uppercase it
  uppercase_attr = {}
  for name, val in future_class_attr.items():
      if not name.startswith('__'):
          uppercase_attr[name.upper()] = val
      else:
          uppercase_attr[name] = val

  # let `type` do the class creation
  return type(future_class_name, future_class_parents, uppercase_attr)

In [9]:
class Foo(metaclass=upper_attr):
    foo = 1
    bar = 2

In [10]:
Foo.FOO

1

In [11]:
Foo.BAR

2

In [12]:
hasattr(Foo, 'bar')

False

In [13]:
dir(Foo())

['BAR',
 'FOO',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

Metaklasa nie nazywałaby się meta**klasą**, jeśli tego samego nie można by było osiągnąć używając **klasy** zamiast **funkcji**.

In [14]:
# remember that `type` is actually a class like `str` and `int`
# so you can inherit from it
class UpperAttrMetaclass(type): 
    # __new__ is the method called before __init__
    # it's the method that creates the object and returns it
    # while __init__ just initializes the object passed as parameter
    # you rarely use __new__, except when you want to control how the object
    # is created.
    # here the created object is the class, and we want to customize it
    # so we override __new__
    # you can do some stuff in __init__ too if you wish
    # some advanced use involves overriding __call__ as well, but we won't
    # see this
    def __new__(upperattr_metaclass, future_class_name, 
                future_class_parents, future_class_attr):

        uppercase_attr = {}
        for name, val in future_class_attr.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return type.__new__(future_class_name, future_class_parents, uppercase_attr)

In [15]:
class UpperAttrMetaclass(type): 

    def __new__(cls, clsname, bases, dct):

        uppercase_attr = {}
        for name, val in dct.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val

        return super(UpperAttrMetaclass, cls).__new__(cls, 
                clsname, bases, uppercase_attr)

## Metaklasy - raz jeszcze, krócej

1. Przechwytujemy akt tworzenia klasy
2. Modyfikujemy nowo tworzoną klasę
3. Zwracamy (podstawiamy) 

## Zastosowanie metaklas

Metaklasy w Pytonie to czarna magia, jeśli pytasz po co masz to stosować, to oznacza, że na pewno nie powinieneś tego stosować. Przypomnijmy raz jeszcze co powiedział Tim Peters:

*Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don't (the people who actually need them know with certainty that they need them, and don't need an explanation about why).*

Jedynym sensownym użyciem metaklas jest tworzenie API. Raz jeszcze o django ORM:

```
class Book(models.Model):
    name = models.CharField(max_length=50)
    pages = models.IntegerField()

django_book = Book(name='The Django', pages='333')
>>> print django_book.pages
333
```

Zwykle nie ma sensu używać metaklas. Są zbyt skomplikowane. Klasy możemy spokojnie modyfikować z użyciem:

 * dekoratorów klas
 * monkey patchingu

## Moduł abc --- Abstract Base Classes

In [ ]:
from collections.abc import Container

In [ ]:
class MyList(Container):
    pass

In [ ]:
dir(MyList)

In [ ]:
from abc import ABCMeta
from abc import abstractmethod

In [ ]:
class MyInterface(metaclass=ABCMeta):
    @abstractmethod
    def foo(self):
        pass
    
    @abstractmethod
    def boo(self):
        pass

In [ ]:
class MyImpl():
    pass

In [ ]:
MyInterface.register(MyImpl)

In [ ]:
o = MyImpl()

In [ ]:
class MyImpl2(MyInterface):
    pass

In [ ]:
o2 = MyImpl2()

## Zastosowanie metaklas do odrobaczania

http://www.dabeaz.com/py3meta/Py3Meta.pdf    